In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Input zip path in your Drive
zip_path = '/content/drive/MyDrive/Chain Snatching Cleanup.v5i.yolov8.zip'
# Destination folder
extract_path = '/content/drive/MyDrive/ABC'

# Create the folder if not exists
os.makedirs(extract_path, exist_ok=True)

# Extract zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted to {extract_path}")


KeyboardInterrupt: 

In [2]:
!pip install ultralytics --upgrade --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.1 MB/s eta 0:00:00


In [ ]:
train: /content/drive/MyDrive/ABC/train/images
val: /content/drive/MyDrive/ABC/train/images

nc: 3
names: [bikescooter, chain_snatcher, person]


SyntaxError: invalid syntax (ipython-input-3394135869.py, line 1)

In [ ]:
from ultralytics import YOLO
import os
import time

# Terminal voice helper
def say(msg):
    print(f"\n🤖 TERMINAL: {msg}")
    time.sleep(0.5)

# Set paths
data_yaml = '/content/drive/MyDrive/ABC/data.yaml'
save_dir = '/content/drive/MyDrive/ABC/yolov8_runs'
os.makedirs(save_dir, exist_ok=True)

say("Welcome back, commander! 🚀 Let's gear up for training.")
say(f"Dataset locked and loaded from: {data_yaml}")
say(f"Checkpoints will be safely stored in: {save_dir}")

# Load model
say("Loading YOLOv8s... this might take a second ⏳")
model = YOLO('yolov8s.pt')

# Training start
say("All systems green. Starting training mission... 🏋️")
results = model.train(
    data=data_yaml,
    epochs=50,                  # adjust as needed
    imgsz=640,
    project=save_dir,
    name='chain-snatching-exp1',
    save=True,
    save_period=1,
    verbose=True,
)

say("Training complete! 🎉")
say("Check your weights inside the runs folder. Best model = 'best.pt' 💎")
say("Ready for the next move, commander? 🚀")



🤖 TERMINAL: Welcome back, commander! 🚀 Let's gear up for training.

🤖 TERMINAL: Dataset locked and loaded from: /content/drive/MyDrive/ABC/data.yaml

🤖 TERMINAL: Checkpoints will be safely stored in: /content/drive/MyDrive/ABC/yolov8_runs

🤖 TERMINAL: Loading YOLOv8s... this might take a second ⏳

🤖 TERMINAL: All systems green. Starting training mission... 🏋️
Ultralytics 8.3.188 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/ABC/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v

In [ ]:
from ultralytics import YOLO
import glob
import os
import cv2
import numpy as np

# Paths
model_path = '/content/drive/MyDrive/ABC/yolov8_runs/chain-snatching-exp14/weights/best.pt'
videos_folder = '/content/drive/MyDrive/ABC/unseensnatch'
output_folder = '/content/drive/MyDrive/ABC/inference_results'
os.makedirs(output_folder, exist_ok=True)

# Load model
model = YOLO(model_path)

# Find videos (update extension as needed)
video_files = glob.glob(os.path.join(videos_folder, "*.mp4"))

for video_path in video_files:
    vid_name = os.path.basename(video_path)
    out_dir = os.path.join(output_folder, f"pred_{os.path.splitext(vid_name)[0]}")
    os.makedirs(out_dir, exist_ok=True)

    print(f"Processing: {video_path}")
    results = model.predict(
        source=video_path,
        save=True,
        save_txt=True,
        save_conf=True,
        project=out_dir,
        name="results",    # Save video as results.mp4 in the output dir
        show=False,
        imgsz=640,
        conf=0.25,
        vid_stride=1,
        stream_buffer=True,
        # Use 'mp4' extension for output
    )

    # Move processed video to .mp4 in out_dir
    avi_video = glob.glob(os.path.join(out_dir, "results.avi"))
    if avi_video:  # If an AVI was made, re-save as MP4
        avi_video = avi_video[0]
        mp4_video = avi_video.replace(".avi", ".mp4")
        cap = cv2.VideoCapture(avi_video)
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(mp4_video, fourcc, fps, (width, height))
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
        cap.release()
        out.release()
        os.remove(avi_video)
        print(f"Saved .mp4 processed video to: {mp4_video}")

    # Detection stats for this video
    confs, classes = [], []
    for r in results:
        for box in r.boxes:
            confs.append(float(box.conf))
            classes.append(int(box.cls))
    if confs:
        print(f'- {vid_name}: Avg confidence = {np.mean(confs):.2f}, Detected counts = {np.bincount(classes)}')
    else:
        print(f'- {vid_name}: No detections!')

print("Inference complete! All processed videos are in inference_results/pred_* folders as .mp4 files.")


Processing: /content/drive/MyDrive/ABC/unseensnatch/Chain_Snatching39.mp4

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/826) /content/drive/MyDrive/ABC/unseensnatch/Chain_Snatching39.mp4: 320x640 1 bikescooter, 1 person, 12.4ms
video 1/1 (frame 2/826) /content/drive/MyDrive/ABC/unseensnatch/Chain_Snatching39.mp4: 320x640 1 bikescooter, 1 person, 10.9ms
video 1/1 (frame 3/826) /content/drive/MyDrive/ABC/unseensnatch/Chain_Snatching39.mp4: 320x640 1 bikescooter, 1 person, 15.1ms
v

In [1]:
from ultralytics import YOLO
import os

# Paths
data_yaml = '/content/drive/MyDrive/ABC/data.yaml'
save_dir = '/content/drive/MyDrive/ABC/yolov8_runs'
os.makedirs(save_dir, exist_ok=True)

# Load model (use your latest last.pt)
model = YOLO('/content/drive/MyDrive/ABC/yolov8_runs/chain-snatching-exp14/weights/last.pt')  # Update exp number if needed

# Train with fewer epochs
results = model.train(
    data=data_yaml,
    epochs=30,
    imgsz=640,
    project=save_dir,
    name='chain-snatching-finetune-small',
    save=True,
    save_period=5,
    verbose=True,
    batch=16,
    augment=True,
    optimizer='Adam',
    lr0=0.0001,
    lrf=0.001,
    momentum=0.9,
    warmup_epochs=2,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    label_smoothing=0.1,
    patience=20,
    cos_lr=True,
    amp=True
)

print("TERMINAL: Training complete! Run the next cell for results.")


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/ABC/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/ABC/yolov8_runs/chain-snatching-exp14/weights/last.pt, momentum=0.9, mosaic=1.0, multi_scale

In [3]:
from ultralytics import YOLO

# Load your fine-tuned model
model_path = '/content/drive/MyDrive/ABC/yolov8_runs/chain-snatching-finetune-small/weights/best.pt'
model = YOLO(model_path)

# Re-validate to confirm metrics
metrics = model.val()

# Print everything neatly
print("TERMINAL: Re-validated metrics:")
print(f"Overall Precision: {metrics.box.mp:.3f}")
print(f"Overall Recall: {metrics.box.mr:.3f}")
print(f"Overall mAP@0.5: {metrics.box.map50:.3f}")
print(f"Overall mAP@0.5:0.95: {metrics.box.map:.3f}")
for i, cls in enumerate(metrics.names.values()):
    print(f"{cls} mAP@0.5: {metrics.box.maps[i]:.3f}")

print("TERMINAL: All set! If these match, you're golden. Want to test on an unseen video or tweak further?")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.2 ms, read: 0.1±0.0 MB/s, size: 41.5 KB)
val: Scanning /content/drive/MyDrive/ABC/train/labels.cache... 5063 images, 14 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 5064/5064 6.8Mit/s 0.0s
train: /content/drive/MyDrive/ABC/train/images/11_Speed_mp4-0007_jpg.rf.8760bd7db358b906c989f3f81a95730e.jpg: ignoring corrupt image/label: cannot identify image file '/content/drive/MyDrive/ABC/train/images/11_Speed_mp4-0007_jpg.rf.8760bd7db358b906c989f3f81a95730e.jpg'
train: /content/drive/MyDrive